In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*3. Gene Chunking*
=============
Given a **preannotated** article (check input folder) analyze the **XML** file via `parse_xml_file(xml_file_path)` which returns a dictionary of "Gene Class" from that file  

The Gene Class has the following information:
- self.gene_id: the gene id (from the xml file)
- self.occurences: A list of occurences of the gene in the text. Each occurence is **3-sentences long**.
    - The second sentence contains the explicit mention of the gene.
    - The first and third sentences are the context of the gene mention.
- self.symbol: the gene symbol (from NCBI. Initialized to None)
- self.organism: the organism of the gene (from NCBI. Initialized to None)
- self.full_name: the full name of the gene (from NCBI. Initialized to None. Set to name_from_article if not found in NCBI)
- self.also_known_as: a list of other names of the gene (from NCBI. Initialized to None. Set to name_from_article if not found in NCBI)
- self.name_from_article: the name of the gene as found in the article

Steps:
1. define the class Gene
2. Parse the xml file
3. update the Gene class with the information from NCBI
Now you have a dictionary of Gene classes.~

TIPS:
- control + F "sentence_buffer" to find where you can adjust the buffer

In [2]:
!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00


In [3]:
from lxml import etree
from Bio import Entrez
import re
import time

class Gene:
    def __init__(self, gene_id):
        self.gene_id = gene_id
        self.occurrences = []  # list to store snippet(s) where the gene was mentioned
        self.symbol = None
        self.organism = None
        self.full_name = None
        self.also_known_as = None
        self.name_from_article = None

    def add_occurrence(self, snippet):
        if snippet not in self.occurrences:  # Avoid duplicates
            self.occurrences.append(snippet)

    def set_name_from_article(self, name_from_article):
        """Sets the temporary name of the gene. This is the name accroding to the article"""
        # will be used if the official name is not available
        self.name_from_article = name_from_article

    def get_name_from_article(self):
        return self.name_from_article

    def get_occurrences(self):
        return self.occurrences

    def get_also_known_as(self):
        return self.also_known_as

    def get_gene_id(self):
        return self.gene_id

    def update_info(self, symbol, organism, full_name, also_known_as):
        self.symbol = symbol
        self.organism = organism
        self.full_name = full_name
        self.also_known_as = also_known_as

    def __repr__(self):
        return (f"Gene({self.gene_id})\n"
                f"  Symbol          : {self.symbol}\n"
                f"  Organism        : {self.organism}\n"
                f"  Full Name       : {self.full_name}\n"
                f"  Also Known As   : {self.also_known_as}\n"
                f"  In-text Name    : {self.name_from_article}\n"
                f"  Occurrences     : {self.occurrences}")   # occurrences is a list of 3-sentence snippets

def parse_xml_file(xml_path):
    """Parses the XML file and returns a gene dictionary keyed by gene ID."""
    tree = etree.parse(xml_path)
    root = tree.getroot()
    gene_dict = {}

    for document in root.findall('document'):
        for passage in document.findall('passage'):
            section_type_elem = passage.find("infon[@key='section_type']")
            if section_type_elem is not None and section_type_elem.text.upper() == "METHODS":
                continue  # Skip passages under METHODS
            # Get the full passage text.
            passage_text_elem = passage.find("text")
            passage_text = passage_text_elem.text if passage_text_elem is not None else ""
            # determine the starting offset for this passage.
            passage_offset_elem = passage.find("offset")
            passage_offset = int(passage_offset_elem.text) if passage_offset_elem is not None else 0
            # Split the passage into sentences using regex.
            sentences = re.split(r'(?<=[.!?])\s+', passage_text)
            # Compute start indices for each sentence within the passage text.
            start_indices = []
            current_index = passage_offset
            for sentence in sentences:
                start_indices.append(current_index)
                current_index += len(sentence) + 1  # account for the delimiter space

            processed_ranges = set()  # To track which sentences have already been covered

            # Process each annotation in the passage.
            for annotation in passage.findall('annotation'):
                ann_type = annotation.find("infon[@key='type']")
                if ann_type is not None and ann_type.text == "Gene":
                    gene_id_elem = annotation.find("infon[@key='identifier']")
                    gene_id = gene_id_elem.text if gene_id_elem is not None else None

                    # Extract the gene name from the annotation text. (Temporary name if official name is not available)
                    in_text_gene_name_elem = annotation.find("text")
                    in_text_gene_name = in_text_gene_name_elem.text if in_text_gene_name_elem is not None else None

                    # Extract the annotation location (offset) to find the sentence containing the gene.
                    location_elem = annotation.find("location")
                    ann_offset = int(location_elem.attrib.get('offset', 0)) if location_elem is not None else 0

                    # Determine which sentence contains the annotation based on its offset.
                    sentence_index = None
                    for i, start in enumerate(start_indices):
                        if start <= ann_offset < start + len(sentences[i]):
                            sentence_index = i
                            break

                    # sentence_buffer
                    if sentence_index is not None:
                        start_sentence = max(0, sentence_index - 1)  # one sentence before
                        end_sentence = min(len(sentences), sentence_index + 2)  # one sentence after
                        range_tuple = (start_sentence, end_sentence)
                        if range_tuple in processed_ranges:
                            continue  # Skip duplicate extractions
                        processed_ranges.add(range_tuple)
                        snippet = " ".join(sentences[start_sentence:end_sentence])
                        if gene_id:
                            if gene_id in gene_dict:
                                gene_dict[gene_id].add_occurrence(snippet)
                            else:
                                gene_obj = Gene(gene_id)
                                gene_obj.add_occurrence(snippet)
                                gene_obj.set_name_from_article(in_text_gene_name)       # Temporary name if official name is not available
                                gene_dict[gene_id] = gene_obj
    return gene_dict

def fetch_and_update_gene_info(gene_dict):
    """Retrieves gene information from NCBI and updates the genes in gene_dict."""
    gene_ids = list(gene_dict.keys())
    if gene_ids:
        # Post the gene IDs to NCBI.
        handle = Entrez.epost(db="gene", id=",".join(gene_ids))
        result = Entrez.read(handle)
        handle.close()

        webenv = result["WebEnv"]
        query_key = result["QueryKey"]

        handle = Entrez.esummary(db="gene", webenv=webenv, query_key=query_key)

        record = Entrez.read(handle)
        handle.close()

        for docsum in record["DocumentSummarySet"]["DocumentSummary"]:
            gene_id = docsum.attributes["uid"]
            symbol = docsum.get('NomenclatureSymbol', 'No symbol')
            organism = docsum.get('Organism', {}).get('ScientificName', 'No organism')
            full_name = docsum.get('NomenclatureName', gene_dict[gene_id].get_name_from_article())
            also_known_as = docsum.get('OtherAliases', gene_dict[gene_id].get_name_from_article())

            # if full name and also known as are not available, use the name from the article
            if (full_name == ''):
                full_name = gene_dict[gene_id].get_name_from_article()
            if (also_known_as == ''):
                also_known_as = gene_dict[gene_id].get_name_from_article()

            if gene_id in gene_dict:
                gene_dict[gene_id].update_info(symbol, organism, full_name, also_known_as)
            # Pause briefly to avoid overwhelming NCBI servers.
            time.sleep(0.5)     # 2 requests per second (safe). WITH API KEY, be increased if needed to 10 requests per second.


# Set your email (and API key if available)
Entrez.email = "email here"
# Entrez.api_key = "your_api_key"

# example2.xml takes a while since it has 64 genes.
xml_path = "full_text_annotated_example.xml"
gene_dict = parse_xml_file(xml_path)
fetch_and_update_gene_info(gene_dict)           # Fetch gene information from NCBI (Optional but has good information)
for gene in gene_dict.values():
    print(gene)
    print(f"There are {len(gene.get_occurrences())} occurrences (each are AT MOST 3-setences long SO MAX of {int(3*len(gene.get_occurrences()))} sentences in total) of {gene.get_gene_id()}:{gene.get_name_from_article()}.")
    print("")

Gene(2670)
  Symbol          : GFAP
  Organism        : Homo sapiens
  Full Name       : glial fibrillary acidic protein
  Also Known As   : ALXDRD
  In-text Name    : glial fibrillary acidic protein
  Occurrences     : ['We found that a lectin, Datura stramonium agglutinin, induced irreversible differentiation in C6 glioma cells. The differentiated cells had long processes, a low rate of proliferation and a high content of glial fibrillary acidic protein. When the medium was replaced with Datura stramonium agglutinin-free medium after 1 h, cell proliferation continued to be inhibited.', 'Proliferation of four human glial tumour cells was also inhibited by Datura stramonium agglutinin. Further, these differentiated human glial tumour cells had long processes and a high content of glial fibrillary acidic protein similar to differentiated C6 glioma cells. Taken together, these observations suggest that Datura stramonium agglutinin may be useful as a new therapy for treating glioma withou

# 4. Extract Gene Ontology using LLM

Loading LLM

In [4]:
# packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
# from goatools import obo_parser

In [5]:
print("Loading LLaMA model.")
model_dir = "/content/drive/MyDrive/Llama 3.2-3B-Instruct-model"
device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
import torch
print(torch.cuda.is_available())

model = AutoModelForCausalLM.from_pretrained(model_dir, local_files_only=True).to(device)


Loading LLaMA model.
True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting GO terms

In [45]:
import re
import json
from concurrent.futures import ThreadPoolExecutor

CHUNK_SIZE = 3  # Number of occurrences to combine per request

def extract_go_terms(gene_name, text_chunk):
    """
    Extracts GO terms for a gene using LLaMA.
    Outputs structured JSON:
    [
        {
            "gene": "GENE_NAME",
            "relation": "RELATIONS TO GO TERM",
            "function": "GO TERM"
        }
    ]
    """
    prompt = f"""
    You are extracting Gene Ontology (GO) terms from scientific text.
    Follow these steps:

    1. Identify GO terms explicitly mentioned for the given gene.
    2. Normalize GO terms if an exact synonym exists.
    3. Determine the gene-to-GO relationship between the gene and the GO term. Use relationships like "is_involved_in", "is_associated_with", etc.
    4. **Only** Output the results in JSON format:

    [
        {{
            "gene": "{gene_name}",
            "relationship": "<Relationship>",
            "function": "<GO Function>"
        }}
    ]

    Gene: {gene_name}
    Text: {text_chunk}

    Answer:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=516,
            temperature=0.1,
            top_p=0.95,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    extracted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return extracted_text

In [51]:
def extract_json_part(text):
    """
    Extracts the JSON block after the "Answer:" section in the text.
    Ensures valid JSON is returned.
    """
    match = re.search(r"Answer:\s*(\[.*?\])", text, re.DOTALL)
    if match:
        json_part = match.group(1).strip()
        json_part = fix_json_format(json_part)

        try:
            return json.loads(json_part)
        except json.JSONDecodeError:
            print(f"JSON Decode Error! Attempting Auto-Fix...\n{json_part}")
            fixed_json = fix_json_format(json_part)
            try:
                return json.loads(fixed_json)
            except json.JSONDecodeError:
                print(f"Error: Could not fix JSON:\n{fixed_json}")
                return []
    else:
        print("Error: 'Answer:' section missing")
    return []


def fix_json_format(json_part):
    """
    Fix common issues in JSON formatting, such as missing commas between items in an array.
    """
    json_part = re.sub(r'\}\s*\{', '},{', json_part)  # Ensure objects are correctly separated
    json_part = re.sub(r',\s*]', ']', json_part)  # Remove trailing commas before closing brackets
    json_part = re.sub(r'\]\s*\[', ', ', json_part)  # Merge multiple arrays into one
    return json_part.strip()


def process_occurrences(gene_name, occurrences):
    """
    Processes occurrences in chunks to improve efficiency.
    """
    results = []
    for i in range(0, len(occurrences), CHUNK_SIZE):
        chunk = "\n\n".join(occurrences[i:i + CHUNK_SIZE])  # Combine multiple text blocks
        go_terms = extract_go_terms(gene_name, chunk)
        if go_terms:
            extracted_json = extract_json_part(go_terms)
            if extracted_json:
                results.extend(extracted_json)
            else:
                print(f"Warning: extract_json_part failed for input:\n{occurrences}\n")
    return results

In [52]:
# Process each gene occurrence and extract GO terms
all_results = []
with ThreadPoolExecutor(max_workers=2) as executor:
    futures = []
    for gene in gene_dict.values():
        gene_name = gene.full_name if gene.full_name else gene.name_from_article
        futures.append(executor.submit(process_occurrences, gene_name, gene.occurrences))

    for future in futures:
        all_results.extend(future.result())

# Remove duplicates using a set
unique_results = {(d["gene"], d["relationship"], d["function"]) for d in all_results if "gene" in d and "relationship" in d and "function" in d}

# Convert back to a list of dictionaries
cleaned_results = [{"gene": gene, "relationship": relationship, "function": function} for gene, relationship, function in unique_results]

if cleaned_results:
    with open("cleaned_gene_ontology_results.json", "w") as f:
        json.dump(cleaned_results, f, indent=4)
    print(json.dumps(cleaned_results, indent=4))
else:
    print("Warning: No valid GO terms extracted.")



Error: 'Answer:' section missing
['We found that a lectin, Datura stramonium agglutinin, induced irreversible differentiation in C6 glioma cells. The differentiated cells had long processes, a low rate of proliferation and a high content of glial fibrillary acidic protein. When the medium was replaced with Datura stramonium agglutinin-free medium after 1 h, cell proliferation continued to be inhibited.', 'Proliferation of four human glial tumour cells was also inhibited by Datura stramonium agglutinin. Further, these differentiated human glial tumour cells had long processes and a high content of glial fibrillary acidic protein similar to differentiated C6 glioma cells. Taken together, these observations suggest that Datura stramonium agglutinin may be useful as a new therapy for treating glioma without side effects.', 'Recently, we found that DSA induced differentiation of astrocytes. Addition of DSA caused a morphological change from a polygonal shape to a stellate shape with many lo



---
# Can ignore this part for now (embedding)


### 1. download the obo file from the gene ontology consortium https://current.geneontology.org/products/pages/downloads.html

In [ ]:
print("Loading Gene Ontology database...")
GO_OBO_FILE = "/content/drive/MyDrive/goa_human.gaf"    # DESTINATION OF THE GO OBO FILE
# go_dag = obo_parser.GODag(GO_OBO_FILE)

def load_human_go_annotations(filepath=GO_OBO_FILE):
    """Load human-specific GO annotations from the GOA database"""
    go_annotations = {}

    with open(filepath, "r") as f:
        for line in f:
            if line.startswith("!"):  # Ignore header lines
                continue

            columns = line.strip().split("\t")
            if len(columns) < 9:
                continue  # Ensure we have enough columns

            gene_name = columns[2]  # DB_Object_Symbol (gene name)
            go_id = columns[4]  # GO term
            qualifier = columns[3]  # Relationship (enables, involved_in, etc.)
            aspect = columns[8]  # BP, MF, or CC

            if gene_name not in go_annotations:
                go_annotations[gene_name] = {"BP": [], "MF": [], "CC": []}

            if aspect == "P":
                go_annotations[gene_name]["BP"].append((go_id, qualifier))
            elif aspect == "F":
                go_annotations[gene_name]["MF"].append((go_id, qualifier))
            elif aspect == "C":
                go_annotations[gene_name]["CC"].append((go_id, qualifier))

    return go_annotations

# Load species-specific GO annotations
human_go_annotations = load_human_go_annotations()

# Check if insulin is in the dataset
if "INSULIN" in human_go_annotations:
    print("Found INSULIN in GO annotations!")
else:
    print("INSULIN NOT found in GO annotations!")



Loading Gene Ontology database...
INSULIN NOT found in GO annotations!


### load the tokenizer and the model and put it to cuda

In [ ]:
import re  # Import regex for better parsing

def remove_duplicates(go_terms):
    """Remove duplicate GO terms while preserving order"""
    seen = set()
    unique_terms = []
    for term in go_terms:
        if term not in seen:
            seen.add(term)
            unique_terms.append(term)
    return unique_terms

def parse_and_validate_results(result_text, genes):
    """Validate generated GO annotations against GO database"""

    # Detect broken output
    if "[IMPORTANT]" in result_text or "Please provide" in result_text:
        print("⚠️ Warning: The model failed to generate GO terms correctly.")
        return {gene: {"BP": [], "MF": [], "CC": []} for gene in genes}

    validated_results = {gene: {"BP": [], "MF": [], "CC": []} for gene in genes}

    bp_terms = re.findall(r"BP:\s*(GO:\d+ \(.+?\))", result_text)
    mf_terms = re.findall(r"MF:\s*(GO:\d+ \(.+?\))", result_text)
    cc_terms = re.findall(r"CC:\s*(GO:\d+ \(.+?\))", result_text)

    for gene in genes:
        validated_results[gene]["BP"] = remove_duplicates(bp_terms)
        validated_results[gene]["MF"] = remove_duplicates(mf_terms)
        validated_results[gene]["CC"] = remove_duplicates(cc_terms)

    print(f"\n✅ Parsed GO terms for {genes}: {validated_results}")  # Debugging

    return validated_results



# ----------------------------
# Test with the sample article
# ----------------------------
sample_article = "insulin is a hormone that regulates blood sugar levels"
print("\nAnalyzing sample article...")
analysis = analyze_article(sample_article)

if not analysis["genes"]:  # Ensure genes exist before proceeding
    print("No genes/proteins detected.")
else:
    genes = analysis["genes"]
    result_text = analysis["result_text"]

    print("\nResults:")
    print(parse_and_validate_results(result_text, genes))



Analyzing sample article...
Detected entities: ['insulin']

Generated GO Terms:
GO terms extracted from the article excerpt:
    - BP: GO:0008150 (carbohydrate metabolic process)
    - MF: GO:0005518 (protein binding)
    - CC: GO:0005886 (plasma membrane)
    - BP: GO:0006006 (glucose metabolic process)
    - MF: GO:0005543 (insulin receptor binding)
    - MF: GO:0005519 (insulin binding)
    - CC: GO:0009611 (extracellular space)
    - MF: GO:0005515 (transmembrane receptor activity)
    - MF: GO:0005516 (transmembrane receptor activity)
    - MF: GO:0005517 (receptor activity)
    - BP: GO:0008152 (glucose-6-phosphate metabolic process)
    - MF: GO:0005518 (protein binding)
    - MF: GO:0005519 (insulin binding)
    - MF: GO:0005515 (transmembrane receptor activity)
    - MF: GO:0005516 (transmembrane receptor activity)
    - MF: GO:0005517 (receptor activity)
    - BP: GO:0008150 (carbohydrate metabolic process)
    - MF: GO:0005543 (insulin receptor binding)
    - MF: GO:0005518